### TP PRATIQUE SUR LA REGULARISATION

Dans ce TP, nous examinerons la mise en œuvre de différentes techniques de régularisation. Tout d’abord, nous commencerons par la régression linéaire multiple. Pour cela, nous avons besoin de l’environnement python3 avec sci-kit learn et pandas
Premièrement, nous devons importer des bibliothèques dans notre environnement.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


Nous allons utiliser l’ensemble de données de prédiction de Boston. Cet ensemble de données est présent dans le module des ensembles de données de la bibliothèque sklearn (scikit-learn). Nous pouvons importer ce jeu de données comme suit.

In [ ]:
# Loading pre-defined Boston Dataset
boston_dataset = datasets.load_boston()
print(boston_dataset.DESCR)


Nous pouvons conclure de la description ci-dessus que nous avons 13 variables indépendantes et une variable dépendante (prix de la maison). Nous devons maintenant vérifier une corrélation entre variables indépendantes et variable dépendante. Nous pouvons utiliser scatterplot/corrplot pour cela. 

In [ ]:
# Generate scatter plot of independent vs Dependent variable
plt.style.use('ggplot')
fig = plt.figure(figsize = (18, 18))

for index, feature_name in enumerate(boston_dataset.feature_names):
    ax = fig.add_subplot(4, 4, index + 1)
    ax.scatter(boston_dataset.data[:, index], boston_dataset.target)
    ax.set_ylabel('House Price', size = 12)
    ax.set_xlabel(feature_name, size = 12)

plt.show()


Le code ci-dessus produit des diagrammes de dispersion de différentes variables indépendantes avec la variable cible comme indiqué ci-dessous

Nous pouvons observer à partir des graphiques de dispersion ci-dessus que certaines des variables indépendantes ne sont pas très corrélées (positivement ou négativement) avec la variable cible. Ces variables verront leurs coefficients être réduits en régularisation. 

In [ ]:
# Load the dataset into Pandas Dataframe
boston_pd = pd.DataFrame(boston_dataset.data)
boston_pd.columns = boston_dataset.feature_names
boston_pd_target = np.asarray(boston_dataset.target)
boston_pd['House Price'] = pd.Series(boston_pd_target)

# input 
X = boston_pd.iloc[:, :-1]

#output
Y = boston_pd.iloc[:, -1]

print(boston_pd.head())


Maintenant, nous appliquons la division train-test pour diviser l’ensemble de données en deux parties, l’une pour l'apprentissage et l’autre pour le test. Nous utiliserons 25 % des données pour les essais. 

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(boston_pd.iloc[:, :-1], boston_pd.iloc[:, -1], test_size = 0.25)

print("Train data shape of X = % s and Y = % s : "%(x_train.shape, y_train.shape))
print("Test data shape of X = % s and Y = % s : "%(x_test.shape, y_test.shape))

#### Régression multiple (linéaire) 
C’est maintenant le bon moment pour tester les modèles. Nous utiliserons d’abord la régression linéaire multiple. Nous formons le modèle sur les données de formation et calculons le MSE sur test. 

In [ ]:
# Apply multiple Linear Regression Model
lreg = LinearRegression()
lreg.fit(x_train, y_train)

# Generate Prediction on test set
lreg_y_pred = lreg.predict(x_test)

# calculating Mean Squared Error (mse)
mean_squared_error = np.mean((lreg_y_pred - y_test)**2)
print("Mean squared Error on test set : ", mean_squared_error)

# Putting together the coefficient and their corresponding variable names 
lreg_coefficient = pd.DataFrame()
lreg_coefficient["Columns"] = x_train.columns
lreg_coefficient['Coefficient Estimate'] = pd.Series(lreg.coef_)
print(lreg_coefficient)


Traçons un graphique à barres des coefficients ci-dessus en utilisant matplotlib bibliothèque de visualisation.

In [ ]:
# plotting the coefficient score
fig, ax = plt.subplots(figsize =(20, 10))

color =['tab:gray', 'tab:blue', 'tab:orange', 
'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 
'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan', 
'tab:orange', 'tab:green', 'tab:blue', 'tab:olive']

ax.bar(lreg_coefficient["Columns"], 
lreg_coefficient['Coefficient Estimate'], 
color = color)

ax.spines['bottom'].set_position('zero')

plt.style.use('ggplot')
plt.show()


Comme nous pouvons observer que beaucoup de variables ont un coefficient insignifiant, ces coefficients n’ont pas beaucoup contribué au modèle et doivent réguler ou même éliminer certaines de ces variables. 

#### Régression de Ridge : 
Ridge Regression a ajouté un terme dans la fonction d’erreur de moindre carré ordinaire qui régularise la valeur des coefficients des variables. Ce terme est la somme des carrés de coefficient multipliés par le paramètre Le motif de l’ajout de ce terme est de pénaliser la variable correspondant à ce coefficient peu corrélé à la variable cible. Ce terme est appelé régularisation L2. 

In [ ]:
# import ridge regression from sklearn library
from sklearn.linear_model import Ridge

# Train the model 
ridgeR = Ridge(alpha = 1)
ridgeR.fit(x_train, y_train)
y_pred = ridgeR.predict(x_test)

# calculate mean square error
mean_squared_error_ridge = np.mean((y_pred - y_test)**2)
print(mean_squared_error_ridge)

# get ridge coefficient and print them
ridge_coefficient = pd.DataFrame()
ridge_coefficient["Columns"]= x_train.columns
ridge_coefficient['Coefficient Estimate'] = pd.Series(ridgeR.coef_)
print("The value of MSE error and the dataframe with ridge coefficients. ")
print(ridge_coefficient)


In [ ]:
print("The bar plot of above data is: ")

Dans le graphique ci-dessus, nous prenons $\alpha = 1$. 
Regardons un autre graphique à barres avec $\alpha = 10$ 

Comme nous pouvons l’observer à partir des graphiques ci-dessus, l’$\alpha$ aide à régulariser le coefficient et à les faire converger plus rapidement. 

Notez que les graphiques ci-dessus peuvent être trompeurs d’une manière qui montre que certains des coefficients deviennent nuls. En régularisation de **Ridge**, les coefficients ne peuvent jamais être 0, ils sont tout simplement trop petits pour être observés dans les parcelles ci-dessus. 
 

 
#### Régression au lasso : 
La régression de Lasso est similaire à la régression de Ridge sauf que nous ajoutons ici la valeur absolue moyenne des coefficients à la place de la valeur carrée moyenne. Contrairement à la régression de crête, la régression de Lasso peut complètement éliminer la variable en réduisant sa valeur de coefficient à 0. Le nouveau terme que nous avons ajouté à Ordinary Least Square (OLS) est appelé régularisation L1.

In [ ]:
# import Lasso regression from sklearn library
from sklearn.linear_model import Lasso

# Train the model
lasso = Lasso(alpha = 1)
lasso.fit(x_train, y_train)
y_pred1 = lasso.predict(x_test)

# Calculate Mean Squared Error
mean_squared_error = np.mean((y_pred1 - y_test)**2)
print("Mean squared error on test set", mean_squared_error)
lasso_coeff = pd.DataFrame()
lasso_coeff["Columns"] = x_train.columns
lasso_coeff['Coefficient Estimate'] = pd.Series(lasso.coef_)

print("The value of MSE error and the dataframe with Lasso coefficients. :")
print(lasso_coeff)


In [ ]:
print("The bar plot of above coefficients: ")

La régression de Lasso a donné le même résultat que la régression de Ridge, lorsque nous augmentons la valeur d’alpha . Regardons un autre graphique à $\alpha = 10$. 

#### Elastic Net : 
Dans la régularisation Elastic Net, nous avons ajouté les termes L1 et L2 pour obtenir la fonction de perte finale. Cela nous amène à réduire la fonction de perte suivante : 
$$ L_{elastic-Net} \left (  \hat\beta\right )=  \left (\sum \left ( y - x_{i}^{J} \hat{ \beta}  \right ) 2 \right )/2n+ \lambda \left ( (1 - \alpha )/2 *  \sum_{j=1} {m}  \hat{ \beta_{j} {2}} + \alpha *  \sum_{j=1} {m}  \left  |  \hat{ \beta_{j}}  \right  |  \right)$$
où alpha est entre 0 et 1. quand alpha = 1, Il réduit la durée de pénalité à L1 et si $alpha = 0$, il réduit ce terme à L2 
pénalité.

In [ ]:
# import model
from sklearn.linear_model import ElasticNet

# Train the model
e_net = ElasticNet(alpha = 1)
e_net.fit(x_train, y_train)

# calculate the prediction and mean square error
y_pred_elastic = e_net.predict(x_test)
mean_squared_error = np.mean((y_pred_elastic - y_test)**2)
print("Mean Squared Error on test set", mean_squared_error)

e_net_coeff = pd.DataFrame()
e_net_coeff["Columns"] = x_train.columns
e_net_coeff['Coefficient Estimate'] = pd.Series(e_net.coef_)
e_net_coeff


#### Conclusion : 
L’analyse ci-dessus permet de tirer la conclusion suivante sur les différentes méthodes de régularisation : 
 

    La régularisation est utilisée pour réduire la dépendance à une variable indépendante particulière en ajoutant le terme de pénalité à la fonction Perte. Ce terme empêche les coefficients des variables indépendantes de prendre des valeurs extrêmes. 
     
    Ridge Regression ajoute le terme de pénalité de régularisation L2 à la fonction de perte. Ce terme réduit les coefficients mais ne les rend pas 0 et n’élimine donc pas complètement toute variable indépendante. Il peut être utilisé pour mesurer l’impact des différentes variables indépendantes. 
     
    La régression Lasso ajoute le terme de pénalité de régularisation L1 à la fonction de perte. Ce terme réduit les coefficients et les rend 0, éliminant ainsi complètement la variable indépendante correspondante. Il peut être utilisé pour la sélection des fonctionnalités, etc. 
     
    Elastic Net est une combinaison des deux régularisations ci-dessus. Il contient à la fois la L1 et la L2 comme terme de pénalité. Il fonctionne mieux que Ridge et Lasso régression pour la plupart des cas de test. 